In [38]:
# Essential imports for data parsing analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
import re
import warnings
warnings.filterwarnings('ignore')


import cmc_parser as cp

# Individual file parsing

In [39]:
def parse_bhformation_dat(model_path):

    filepath= model_path+'/initial.bhformation.dat'
    # Define column names for the core data we want to keep
    column_names = [
        'time', 'r', 'binary', 'ID', 'zams_m', 'm_progenitor', 'bh_mass',
        'bh_spin', 'birth_kick'
    ]

    # Read the file, skipping any lines that start with #
    data = []
    with open(filepath, 'r') as file:
        for line in file:
            if not line.startswith('#'):
                # Split the line by whitespace
                values = line.strip().split()
                if len(values) >= 9:  # Ensure we have enough values for our columns
                    # Only take the first 9 values (excluding vs columns)
                    data.append(values[:9])

    # Convert to DataFrame
    df = pd.DataFrame(data, columns=column_names)

    # Convert columns to appropriate data types
    numeric_cols = ['time', 'r', 'zams_m', 'm_progenitor', 'bh_mass', 'bh_spin']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col])

    # Convert binary and ID to integers
    df['binary'] = df['binary'].astype(int)
    df['ID'] = df['ID'].astype(int)

    # Handle birth_kick column
    df['birth_kick'] = pd.to_numeric(df['birth_kick'], errors='coerce')

    return df
parse_bhformation_dat('data/N2.0rv0.5rg2.0Z0.0002')

,time,r,binary,ID,zams_m,m_progenitor,bh_mass,bh_spin,birth_kick
0,0.009197,0.190420,1,124149,0.000000,47.24320,42.51890,0,0.0000
1,0.009476,0.784866,0,178115,104.318000,101.11100,40.50000,0,1.0000
2,0.009920,0.069105,1,150155,0.000000,37.76810,33.99130,0,0.0000
3,0.010120,0.109359,1,228329,0.000000,35.36190,31.82570,0,0.0000
4,0.010364,1.755900,0,9114,0.181263,32.65030,29.38530,0,1.0000
...,...,...,...,...,...,...,...,...,...
338,0.039135,0.127507,1,216993,0.000000,75.53730,67.98360,0,0.0000
339,0.040909,0.233458,0,185677,0.178679,25.05900,9.83723,0,151.7321
340,0.043852,0.321977,0,238752,0.336246,21.79050,19.61150,0,1.0000
341,6.507828,0.035967,1,0,0.000000,4.23882,16.73940,0,0.0000


In [40]:
def parse_collision_log(model_path):

    filepath= model_path +'/initial.collision.log'
    data = {
        'time': [],
        'collision_type': [],
        'idm': [],
        'mm': [],
        'id1': [],
        'm1': [],
        'id2': [],
        'm2': [],
        'r': [],
        'typem': [],
        'type1': [],
        'type2': []
    }

    with open(filepath, 'r') as file:
        for line in file:
            if line.startswith('t='):
                # Extract time
                time_match = re.search(r't=([\d.eE+-]+)', line)
                # Extract collision type (binary-single, binary-binary, single-single)
                type_match = re.search(r'\s(binary-[a-z]+|single-single)\s', line)
                # Extract idm and mm
                idm_mm_match = re.search(r'idm=(\d+)\(mm=([\d.eE+-]+)\)', line)
                # Extract id1 and m1
                id1_m1_match = re.search(r'id1=(\d+)\(m1=([\d.eE+-]+)\)', line)
                # Extract id2 and m2
                id2_m2_match = re.search(r'id2=(\d+)\(m2=([\d.eE+-]+)\)', line)
                # Extract r
                r_match = re.search(r'\(r=([\d.eE+-]+)\)', line)
                # Extract typem, type1, type2
                typem_match = re.search(r'typem=(\d+)', line)
                type1_match = re.search(r'type1=(\d+)', line)
                type2_match = re.search(r'type2=(\d+)', line)

                if all([time_match, type_match, idm_mm_match, id1_m1_match, id2_m2_match, r_match, typem_match, type1_match, type2_match]):
                    data['time'].append(float(time_match.group(1)))
                    data['collision_type'].append(type_match.group(1))
                    data['idm'].append(int(idm_mm_match.group(1)))
                    data['mm'].append(float(idm_mm_match.group(2)))
                    data['id1'].append(int(id1_m1_match.group(1)))
                    data['m1'].append(float(id1_m1_match.group(2)))
                    data['id2'].append(int(id2_m2_match.group(1)))
                    data['m2'].append(float(id2_m2_match.group(2)))
                    data['r'].append(float(r_match.group(1)))
                    data['typem'].append(int(typem_match.group(1)))
                    data['type1'].append(int(type1_match.group(1)))
                    data['type2'].append(int(type2_match.group(1)))

    df = pd.DataFrame(data)
    return df
parse_collision_log('data/N2.0rv0.5rg2.0Z0.0002')

,time,collision_type,idm,mm,id1,m1,id2,m2,r,typem,type1,type2
0,0.000075,single-single,224195,1.74415,14195,0.362653,17005,1.381500,0.285404,1,0,1
1,0.001214,single-single,312311,75.03210,102311,74.907900,123824,0.124205,0.505865,1,1,0
2,0.001258,binary-single,221957,39.52030,11957,0.163859,50618,39.356400,0.493831,1,0,1
3,0.002027,binary-single,238748,54.47040,81217,0.210071,28748,54.260300,0.122047,1,0,1
4,0.002727,binary-binary,223426,86.49350,13426,0.660127,147898,85.833300,0.087544,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1345,9.206720,single-single,223184,2.09738,13184,1.094970,85418,1.002410,0.044494,6,1,11
1346,9.211420,single-single,270659,1.53971,60659,0.990716,101242,1.081490,0.039341,4,1,4
1347,9.230450,binary-single,278553,5.27436,68552,1.152800,303927,4.122100,0.014885,3,3,1
1348,9.231720,binary-single,216287,4.88003,6286,1.157010,278553,5.274300,0.015638,5,11,3


In [41]:
def parse_merger_file(model_path):
    filepath = model_path + '/initial.semergedisrupt.log'
    # Initialize lists to store the extracted data
    data = {
        'time': [],
        'interaction_type': [],
        'id_rem': [],
        'mass_rem': [],
        'id1': [],
        'm1': [],
        'id2': [],
        'm2': [],
        'r': [],
        'type_rem': [],
        'type1': [],
        'type2': []
    }

    with open(filepath, 'r') as file:
        for line in file:
            # Skip comment lines
            if line.startswith('#'):
                continue

            # Check if this is a data line
            if line.startswith('t='):
                # Extract time
                time_match = re.search(r't=([0-9.eE+-]+)', line)
                time = float(time_match.group(1)) if time_match else np.nan

                # Extract interaction type
                type_match = re.search(r't=[\d.eE+-]+\s+(\w+)', line)
                interaction_type = type_match.group(1) if type_match else np.nan

                # Extract remnant ID and mass
                id_rem_match = re.search(r'idr=(\d+)\(mr=([\d.eE+-]+)\)', line)
                id_rem = int(id_rem_match.group(1)) if id_rem_match else np.nan
                mass_rem = float(id_rem_match.group(2)) if id_rem_match else np.nan

                # Extract id1 and m1
                id1_m1_match = re.search(r'id1=(\d+)\(m1=([\d.eE+-]+)\)', line)
                id1 = int(id1_m1_match.group(1)) if id1_m1_match else np.nan
                m1 = float(id1_m1_match.group(2)) if id1_m1_match else np.nan

                # Extract id2 and m2
                id2_m2_match = re.search(r'id2=(\d+)\(m2=([\d.eE+-]+)\)', line)
                id2 = int(id2_m2_match.group(1)) if id2_m2_match else np.nan
                m2 = float(id2_m2_match.group(2)) if id2_m2_match else np.nan

                # Extract radius
                r_match = re.search(r'\(r=([\d.eE+-]+)\)', line)
                r = float(r_match.group(1)) if r_match else np.nan

                # Extract types
                type_rem_match = re.search(r'typer=(\d+)', line)
                type1_match = re.search(r'type1=(\d+)', line)
                type2_match = re.search(r'type2=(\d+)', line)

                type_rem = int(type_rem_match.group(1)) if type_rem_match else np.nan
                type1 = int(type1_match.group(1)) if type1_match else np.nan
                type2 = int(type2_match.group(1)) if type2_match else np.nan

                # Append data
                data['time'].append(time)
                data['interaction_type'].append(interaction_type)
                data['id_rem'].append(id_rem)
                data['mass_rem'].append(mass_rem)
                data['id1'].append(id1)
                data['m1'].append(m1)
                data['id2'].append(id2)
                data['m2'].append(m2)
                data['r'].append(r)
                data['type_rem'].append(type_rem)
                data['type1'].append(type1)
                data['type2'].append(type2)

    # Create DataFrame
    df = pd.DataFrame(data)
    return df

# Usage
merger_df = parse_merger_file('data/N2.0rv0.5rg2.0Z0.0002')

merger_df


,time,interaction_type,id_rem,mass_rem,id1,m1,id2,m2,r,type_rem,type1,type2
0,0.001989,disrupt1,312311.0,75.21870,312311,74.964600,162670,0.254204,0.314027,1.0,1,0
1,0.005335,disrupt1,136651.0,89.76560,136651,89.665800,125486,0.099913,0.221251,1.0,1,0
2,0.005919,disrupt1,161709.0,83.86030,161709,78.209600,204653,5.650770,0.063689,1.0,1,1
3,0.007645,disrupt1,325918.0,78.02040,325918,77.910400,158314,0.110015,0.062531,1.0,1,0
4,0.008486,disrupt2,175727.0,109.86200,161895,0.092841,175727,109.886000,0.181820,2.0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
618,9.037510,disruptboth,NaN,NaN,210009,1.664140,160681,1.102050,0.012559,NaN,8,1
619,9.157400,disrupt2,287355.0,2.35583,155861,0.980142,287355,1.870000,0.017850,5.0,11,6
620,9.162860,disrupt2,378095.0,2.80844,162500,0.876363,378095,2.070240,0.008220,4.0,11,5
621,9.181540,disruptboth,NaN,NaN,219323,1.107870,125797,1.202410,0.011862,NaN,5,11


# Unit Conversion

In [42]:
def load_conversion_factors(model_path):

    
    conv_file_path = os.path.join(model_path, 'initial.conv.sh')

    # Load conversion factors using cmc_parser - exactly as you showed
    conv_file = cp.conversion_file(conv_file_path)
        
    # Extract the conversion factors you demonstrated
    conversion_factors = {
        'mass_msun_mstar': conv_file.mass_msun_mstar,  # For converting masses
        'time_myr': conv_file.time_myr,                # For converting times 
        'length_parsec': conv_file.length_parsec,      # For converting radii
        'conv_obj': conv_file                          # Store full object for reference
    }
        
    return conversion_factors

# discover models

In [43]:

def parse_model_name(model_name: str):
    """
    Extract N, rv, rg and Z from directory names such as
    'N0.8rv0.75rg20Z0.002'  or  'N8e5rv0.75rg20z0.002_extra'.
    """
    # digits, dots, +, -, or scientific-notation letters
    num = r'([0-9.eE+-]+)'

    # ^ anchors at start; we don’t anchor at end so trailing text is allowed
    pattern = rf'^N{num}rv{num}rg{num}Z{num}'

    m = re.match(pattern, model_name, flags=re.IGNORECASE)
    if not m:
        # keep the warning so you can still see badly-named folders
        print(f"Warning: Could not parse {model_name}")
        return None

    return {
        'N':  float(m.group(1)),
        'rv': float(m.group(2)),
        'rg': float(m.group(3)),
        'Z':  float(m.group(4)),
        'model_name': model_name
    }

In [46]:
def discover_models(data_directory):
    # Need to implement filtering
    model_dirs=[]
    for item in os.listdir(data_directory):
        item_path = os.path.join(data_directory, item)
        if os.path.isdir(item_path):
            parsed = parse_model_name(item)
            if parsed:
                parsed['path'] = item_path
                model_dirs.append(parsed)
        models_df = pd.DataFrame(model_dirs)
    return models_df

model_list = discover_models('data')
model_list

,N,rv,rg,Z,model_name,path
0,16.0,0.5,2.0,0.0020,N16rv0.5rg2.0Z0.002,data\N16rv0.5rg2.0Z0.002
1,16.0,0.5,2.0,0.0200,N16rv0.5rg2.0Z0.02,data\N16rv0.5rg2.0Z0.02
2,16.0,0.5,20.0,0.0020,N16rv0.5rg20.0Z0.002,data\N16rv0.5rg20.0Z0.002
3,16.0,0.5,20.0,0.0200,N16rv0.5rg20.0Z0.02,data\N16rv0.5rg20.0Z0.02
4,16.0,0.5,8.0,0.0020,N16rv0.5rg8.0Z0.002,data\N16rv0.5rg8.0Z0.002
...,...,...,...,...,...,...
140,8.0,4.0,20.0,0.0020,N8.0rv4.0rg20.0Z0.002,data\N8.0rv4.0rg20.0Z0.002
141,8.0,4.0,20.0,0.0200,N8.0rv4.0rg20.0Z0.02,data\N8.0rv4.0rg20.0Z0.02
142,8.0,4.0,8.0,0.0002,N8.0rv4.0rg8.0Z0.0002,data\N8.0rv4.0rg8.0Z0.0002
143,8.0,4.0,8.0,0.0020,N8.0rv4.0rg8.0Z0.002,data\N8.0rv4.0rg8.0Z0.002


In [166]:
def full_model(model_path,latest=False):
    #getting the frames
    all_bh=parse_bhformation_dat(model_path)
    all_col= parse_collision_log(model_path)
    all_merge=parse_merger_file(model_path)

    # unit conversion
    conv_factors=load_conversion_factors(model_path)
    myr = conv_factors['time_myr']
    parsec = conv_factors['length_parsec']
    msun = conv_factors['mass_msun_mstar']

    #colision units

    all_col['m1']=all_col['m1']*msun # convert to m sun
    all_col['m2']=all_col['m2']*msun # convert to m sun
    all_col['mm']=all_col['mm']*msun # convert to m sun
    all_col['time']=all_col['time']*myr# convert to MYr
    all_col['r']=all_col['r']*parsec # convert to Parsec
    
    #all bh units
    all_bh['time']=all_bh['time']*myr# convert to MYr
    all_bh['r']=all_bh['r']*parsec # convert to Parsec
    all_bh['bh_mass']=all_bh['bh_mass']*msun # convert to m sun
    all_bh['m_progenitor']=all_bh['m_progenitor']*msun # convert to m sun
    all_bh['zams_m']=all_bh['zams_m']*msun # convert to m sun
    
    #all merge units
    all_merge['m1']=all_merge['m1']*msun # convert to m sun
    all_merge['m2']=all_merge['m2']*msun # convert to m sun
    all_merge['time']=all_merge['time']*myr# convert to MYr
    all_merge['r']=all_merge['r']*parsec # convert to Parsec
    all_merge['mass_rem']=all_merge['mass_rem']*msun # convert to m sun

    # Collision and Merge BHs 
    col_bh = pd.merge(all_col,all_bh,left_on='idm',right_on='ID',how='inner')
    merge_bh = pd.merge(all_merge,all_bh,left_on='id_rem',right_on='ID',how='inner')

    # model path tagging
    all_bh['model'] = model_path
    all_col['model'] = model_path
    all_merge['model'] = model_path
    col_bh['model'] = model_path
    merge_bh['model'] = model_path
    
    #latest event merger fix

    if latest == True:
        filtered_df = merge_bh[merge_bh['time_y'] >= merge_bh['time_x']]

        merge_bh = filtered_df.loc[filtered_df.groupby('ID')['time_x'].idxmax()]

    #output

    all_frames = {'all_bh':all_bh,'col':all_col,'merge':all_merge,'col_bh':col_bh,'merge_bh':merge_bh}
    return all_frames



In [169]:
mtest=full_model(model_list['path'][0])['merge_bh']
mtest.groupby('id_rem').count().sort_values('time_x',ascending=False).head(10)



,time_x,interaction_type,mass_rem,id1,m1,id2,m2,r_x,type_rem,type1,...,time_y,r_y,binary,ID,zams_m,m_progenitor,bh_mass,bh_spin,birth_kick,model
id_rem,,,,,,,,,,,,,,,,,,,,,
1690159.0,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,11,11,11,11,11,11
1693652.0,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
206906.0,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
1692519.0,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
325363.0,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
1706983.0,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1738745.0,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1744502.0,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1723701.0,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [156]:
mtest[mtest['id_rem']==325363.0		]


,time_x,interaction_type,id_rem,mass_rem,id1,m1,id2,m2,r_x,type_rem,...,time_y,r_y,binary,ID,zams_m,m_progenitor,bh_mass,bh_spin,birth_kick,model
5,2.903005,disrupt1,325363.0,49.408839,325363,37.370760,1628074,12.038140,0.036610,1.0,...,6.777374,0.108362,1,325363,0.0,17.085584,15.377043,0,0.0,data\N16rv0.5rg2.0Z0.002
56,6.423973,disrupt1,325363.0,45.159838,325363,45.876708,894274,0.099115,0.113643,2.0,...,6.777374,0.108362,1,325363,0.0,17.085584,15.377043,0,0.0,data\N16rv0.5rg2.0Z0.002
146,38.993982,disrupt1,325363.0,38.889833,325363,15.377043,1791326,25.281797,0.008676,14.0,...,6.777374,0.108362,1,325363,0.0,17.085584,15.377043,0,0.0,data\N16rv0.5rg2.0Z0.002
147,41.657393,disrupt1,325363.0,80.539313,325363,38.889833,1691443,46.247515,0.020821,14.0,...,6.777374,0.108362,1,325363,0.0,17.085584,15.377043,0,0.0,data\N16rv0.5rg2.0Z0.002


In [162]:
filtered_df = mtest[mtest['time_y'] >= mtest['time_x']]

mlatest = filtered_df.loc[filtered_df.groupby('ID')['time_x'].idxmax()]
mlatest[mlatest['id_rem']==325363.0		]


,time_x,interaction_type,id_rem,mass_rem,id1,m1,id2,m2,r_x,type_rem,...,time_y,r_y,binary,ID,zams_m,m_progenitor,bh_mass,bh_spin,birth_kick,model
56,6.423973,disrupt1,325363.0,45.159838,325363,45.876708,894274,0.099115,0.113643,2.0,...,6.777374,0.108362,1,325363,0.0,17.085584,15.377043,0,0.0,data\N16rv0.5rg2.0Z0.002


In [170]:
btest=full_model(model_list['path'][0])['all_bh']
ctest=full_model(model_list['path'][0])['col_bh']
btest.groupby('ID').count().sort_values('time')


,time,r,binary,zams_m,m_progenitor,bh_mass,bh_spin,birth_kick,model
ID,,,,,,,,,
2936673,1,1,1,1,1,1,1,1,1
2808916,1,1,1,1,1,1,1,1,1
2814097,1,1,1,1,1,1,1,1,1
2815387,1,1,1,1,1,1,1,1,1
2815998,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...
1699097,2,2,2,2,2,2,2,2,2
1722330,2,2,2,2,2,2,2,2,2
1720888,2,2,2,2,2,2,2,2,2


In [171]:
btest[btest['ID']==1814648]

,time,r,binary,ID,zams_m,m_progenitor,bh_mass,bh_spin,birth_kick,model
234,4.610939,0.070344,0,1814648,0.274771,14.165400,12.748823,0,1.0,data\N16rv0.5rg2.0Z0.002
2714,13.226412,0.148394,0,1814648,0.048652,9.467051,8.504917,0,1.0,data\N16rv0.5rg2.0Z0.002


In [172]:
ctest[ctest['idm']==1814648.0]

,time_x,collision_type,idm,mm,id1,m1,id2,m2,r_x,typem,...,time_y,r_y,binary,ID,zams_m,m_progenitor,bh_mass,bh_spin,birth_kick,model
250,4.399939,single-single,1814648,17.036215,1814647,16.800898,1498901,0.242156,0.103385,4,...,4.610939,0.070344,0,1814648,0.274771,14.165400,12.748823,0,1.0,data\N16rv0.5rg2.0Z0.002
251,4.399939,single-single,1814648,17.036215,1814647,16.800898,1498901,0.242156,0.103385,4,...,13.226412,0.148394,0,1814648,0.048652,9.467051,8.504917,0,1.0,data\N16rv0.5rg2.0Z0.002
1384,8.202169,single-single,1814648,26.929870,134648,26.845265,298135,0.084583,0.058128,1,...,4.610939,0.070344,0,1814648,0.274771,14.165400,12.748823,0,1.0,data\N16rv0.5rg2.0Z0.002
1385,8.202169,single-single,1814648,26.929870,134648,26.845265,298135,0.084583,0.058128,1,...,13.226412,0.148394,0,1814648,0.048652,9.467051,8.504917,0,1.0,data\N16rv0.5rg2.0Z0.002


In [163]:
mtest[mtest['id_rem']==298135.0		]

,time_x,interaction_type,id_rem,mass_rem,id1,m1,id2,m2,r_x,type_rem,...,time_y,r_y,binary,ID,zams_m,m_progenitor,bh_mass,bh_spin,birth_kick,model


In [165]:
btest[btest['ID']==298135]

,time,r,binary,ID,zams_m,m_progenitor,bh_mass,bh_spin,birth_kick,model


In [89]:
# after we confirm this works we can implement filtering and convert to a function

# agregating models

all_paths=model_list['path'][:2]
# Initialize empty lists to collect DataFrames for each key
all_bh_list = []
merge_list = []
col_list = []
merge_bh_list = []
col_bh_list = []

# Loop through all paths and collect DataFrames
for path in all_paths:
    model = full_model(path)
    all_bh_list.append(model['all_bh'])
    merge_list.append(model['merge'])
    col_list.append(model['col'])
    merge_bh_list.append(model['merge_bh'])
    col_bh_list.append(model['col_bh'])

# Now concatenate each list of DataFrames into single DataFrames
ag_all_bh = pd.concat(all_bh_list, ignore_index=True)
ag_merge = pd.concat(merge_list, ignore_index=True)
ag_col = pd.concat(col_list, ignore_index=True)
ag_merge_bh = pd.concat(merge_bh_list, ignore_index=True)
ag_col_bh = pd.concat(col_bh_list, ignore_index=True)




In [90]:
print(ag_col_bh)

            time_x collision_type      idm         mm      id1         m1  \
0         1.031471  single-single  1782184  14.837511   232722  14.638643   
1         1.625898  single-single  1869642  14.062297   189642   0.076694   
2         1.676013  single-single  1936824  13.430638   256824   0.547746   
3         1.758030  single-single  1740170  27.581603    60170  27.284060   
4         1.872144  binary-single  1717142  37.700812   161043   0.055559   
...            ...            ...      ...        ...      ...        ...   
4871  13777.862580  binary-binary        0   1.275768  2353668   0.702363   
4872  13777.862580  binary-binary        0   1.275768  2353668   0.702363   
4873  13777.862580  binary-binary        0   1.275768  2353668   0.702363   
4874  13777.862580  binary-binary        0   1.275768  2353668   0.702363   
4875  13777.862580  binary-binary        0   1.275768  2353668   0.702363   

          id2         m2       r_x  typem  ...     time_y       r_y  binary